<a href="https://colab.research.google.com/github/AfsanehHabibi/reddit-conversation-quality/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 1.3 MB/s eta 0:00:00


In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=4d9c0f01b12eaca0963817cd9aedd3dad5f2adebe0a61e9e1e7795a0b3b04d4b
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [31]:
!pip install  kili

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.9/209.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
  Created wheel for cuid: filename=cuid-0.4-py2.py3-none-any.whl size=4712 sha256=2c18a4d57cca91891f3e766467bb872e29e403eae6a974f6777490a0923d0190
  Stored in directory: /root/.cache/pip/wheels/52/0a/dc/a0e28c435d5a74d9eef3d7c3cd147b96cb21e71e5ec7dcfdbe
Successfully built cuid
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [ ]:
#!pip3 install convokit
#!python3 -m spacy download en

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Preprocessing

In [4]:
import numpy as np
import json
from anytree import Node, RenderTree

In [5]:
base_path = "/content/drive/MyDrive/University/RedditData/"

In [6]:
def extract_files_list(file_list_path):
    files_name = []
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                files_name.append(file_path)
    return files_name

In [7]:
data_files_name = extract_files_list(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")

In [8]:
def make_submission_file_path(file_name):
    return f"{base_path}submissions_{file_name}.json"

In [9]:
def make_comment_file_path(file_name):
    return f"{base_path}comments_{file_name}.json"

In [10]:
subreddits = set()

In [11]:
def extract_submissions_text_with_filter(data_files_name):
    submissions_dic = {}
    over_18_count = 0
    for file_name in data_files_name:
        file_path = make_submission_file_path(file_name)
        with open(file_path, 'r') as file:
            lines_sub = file.readlines()
        for l in lines_sub:
            obj = json.loads(l)
            if obj['over_18'] == False:
                submissions_dic[obj['id']]  = obj['title'] + obj['selftext']
                subreddits.add(obj['subreddit'])
            else:
                over_18_count += 1
    print("over 18 count ", over_18_count)
    return submissions_dic

In [12]:
subsmissions = extract_submissions_text_with_filter(data_files_name)

over 18 count  66876


In [13]:
len(subsmissions)

96333

In [14]:
import json

def count_comments_in_file(file_path):
    count = 0
    with open(file_path, 'r') as json_file:
        lines = json_file.readlines()
        for line in lines:
            data = json.loads(line)
            count += len(data.get('comments', []))
    return count

def total_comments_in_files(file_list_path):
    total_comments = 0
    with open(file_list_path, 'r') as file_list:
        for line in file_list:
            print(line)
            line = line.strip()
            if line:
                file_path, _ = line.split(',')
                total_comments += count_comments_in_file(f"{base_path}comments_{file_path}.json")
    return total_comments

# Replace 'file_list.txt' with the path to your text file containing the list of file names
#total_comments = total_comments_in_files(f"{base_path}timestamps_seed2_num20_period10_date2022-10-1.txt_comments_progress.txt")
#print(f'Total number of comments in the files: {total_comments}')

In [15]:
def load_comments_with_filter(files_name, submmissions):
    # Create a dictionary to store the comment nodes by their IDs
    conversations_dic = {}
    for file_name in data_files_name:
        comment_file = make_comment_file_path(file_name)
        with open(comment_file, 'r') as file:
            lines = file.readlines()

        for line in lines:
            comment_nodes = {}
            json_obj = json.loads(line)
            submission_id = json_obj['submission_id']
            comments = json_obj['comments']
            if len(comments) == 0:
              continue
            if submission_id not in subsmissions:
                #submission has removed in preprocessing
                continue
            flat_comments_list = []
            parent_child_pairs = []
            # Create root node for the submission
            submission_text = subsmissions[submission_id]
            root = Node(submission_id, body = submission_text)
            comment_nodes[submission_id] = root
            conversation_text = submission_text
            flat_comments_list.append({'id': submission_id, 'body': submission_text, 'author':None, 'reply_to':None, 'conversation_id':submission_id})
            # Create child nodes for the comments
            for comment in comments:
                # Check if comment is from a bot and ignore it if it is
                if "I am a bot, and this action was performed automatically" in comment['body']:
                    continue

                comment_id = comment['id']
                author = comment.get('author_fullname', 'unknown')
                parent_id = comment['parent_id'].split('_', 1)[-1]
                body = comment['body']
                comment_node = Node(comment_id, body=body)

                if parent_id in comment_nodes:
                    flat_comments_list.append({'id': comment_id, 'body': body, 'author':author, 'reply_to':parent_id, 'conversation_id':submission_id})
                    parent_node = comment_nodes[parent_id]
                    parent_child_pairs.append({
                        'comment': parent_node.body,
                        'reply': body,
                        'comment_id': parent_node.name,
                        'reply_id': comment_id,
                        'global_id': parent_node.name + "_" + comment_id
                    })
                    comment_node.parent = parent_node
                    conversation_text += "/n"+ body
                    comment_nodes[comment_id] = comment_node
            # Print the tree structure
            # print(RenderTree(root))
            conversations_dic[submission_id] = {
                'tree': root,
                'full_text': conversation_text,
                'pairs': parent_child_pairs,
                'comments': flat_comments_list
            }
            #for parent in root.children:
            #  for child in parent.children:
            #       parent_child_pairs.append((parent.body, child.body))

            # Print the parent-child pairs
            #for pair in parent_child_pairs:
            #   print(f"Parent: {pair[0]}, Child: {pair[1]}")

            # Print the tree structure
            #for pre, fill, node in RenderTree(root):
            #   print(f"{pre}{node.name}")

        #print(RenderTree(conversations_tree[2555]))
    return conversations_dic

In [16]:
conversations_dic = load_comments_with_filter(data_files_name, subsmissions)

In [17]:
len(conversations_dic)

39784

In [18]:
def filter_empty_conversations(conversations_dic):
  non_empty_conversations = {}
  for id in conversations_dic:
    new_pairs = []
    pairs = conversations_dic[id]['pairs']
    for pair in pairs:
      if not(pair['comment'] == '[deleted]' or pair['reply'] == '[deleted]' or pair['comment'].endswith('[removed]')):
        new_pairs.append(pair)
    if len(new_pairs) > 0:
      conversations_dic[id]['pairs'] = new_pairs
      non_empty_conversations[id] = conversations_dic[id]
  return non_empty_conversations


In [19]:
non_empty_conversations = filter_empty_conversations(conversations_dic)

In [20]:
len(conversations_dic)

39784

In [21]:
len(non_empty_conversations)

35393

In [22]:
from langdetect import detect, LangDetectException

def filter_non_english(conversations_dic):
    english_conversations = {}
    for id in conversations_dic:
        try:
            language = detect(conversations_dic[id]['full_text'])
            if language == 'en':
                english_conversations[id] = conversations_dic[id]
        except LangDetectException as e:
            # Handle the exception and print the problematic input
            # print(f"Language detection error for comment: {conversations_dic[id]['full_text']}")
            continue
    return english_conversations

In [23]:
english_conversations = filter_non_english(non_empty_conversations)

In [24]:
len(english_conversations)

33566

In [25]:
import re

def contains_question(text):
    # Check if the text contains a question mark
    if '?' in text:
        return True

    # Check if the text contains certain question words
    question_words = ['what', 'how', 'when', 'where', 'who', 'why']
    for word in question_words:
        if re.search(r'\b{}\b'.format(word), text, re.IGNORECASE):
            return True

    # Check if the text contains the word "please"
    if re.search(r'\bplease\b', text, re.IGNORECASE):
        return True

    # If none of the above conditions are met, return False
    return False

# Example usage
text = "What is the capital of France? I love Paris. How do you make a cake? The recipe is easy to follow. Can you please pass the salt?"
if contains_question(text):
    print("The text contains a question.")
else:
    print("The text does not contain a question.")

The text contains a question.


In [26]:
import random

random_pair = random.choice(list(english_conversations.items()))
for pair in random_pair[1]['pairs']:
  print(contains_question(pair['comment']))
  print("comment:", pair['comment'])
  print("reply:", pair['reply'])


True
comment: Meet up before the show?Hey, guys! Very much looking forward to these Red Rocks shows! I know it's been asked a lot on here already but would anyone be interested in meeting up for a drink on Saturday or Monday before the show? 

I work a ton and won't be able to make it to the meetup on Sunday...
reply: Yes! I’ll PM you!!


In [27]:
import random


# Define the sample size (e.g., 3 for a sample of 3 elements)
sample_size = 100
random_pairs = []

# Use random.sample to get a random sample of the specified size
random_sample = random.sample(english_conversations.items(), sample_size)
for sample_conversation in random_sample:
    random_pair = random.choice(sample_conversation[1]['pairs'])
    random_pairs.append(random_pair)

<ipython-input-27-0df81c083ef2>:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_sample = random.sample(english_conversations.items(), sample_size)


In [28]:
len(random_pairs)

100

In [29]:
random_pairs[:5]

[{'comment': 'Does not make it okay, does it?',
  'reply': '“it’s not okay.” is about the worst argument that anyone can come up with in relation to literally any problem.',
  'comment_id': 'iqz6fvw',
  'reply_id': 'iqz94rp',
  'global_id': 'iqz6fvw_iqz94rp'},
 {'comment': 'TrueNAS + Plex: How do get Plex to see my files?Apologies if this is not the correct subreddit.\n\nI have Plex set up through my TrueNAS server. I have two pools - GLaDOS and tank. GLaDOS holds the media files (movies ,tv shows, music), and tank holds the jail for Plex.\n\nI have mount points set up to create a link between tank and GLaDOS so that way Plex sees the folders for the libraries. I also had to give full access to @everyone in order for this to happen.\n\nNow, Plex sees the media folders- and it sees the files inside of the folders, but when I perform a library scan - it does not populate the Plex files in the library.\nHere is an [imgur album](https://imgur.com/a/NurYIOc) of what I am describing. \n\nHow

In [32]:
import os

from kili.client import Kili
kili = Kili(api_key='6acd489f-9746-4ac4-9650-54e2d9c9faba')
interface = {
    "jobs": {
        "JOB_0": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"Related": {"name": "is relevent"}, "UnRelated": {"name": "is not relevent"}},
                "input": "radio",
            },
        },
        "JOB_1": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"HasQuality": {"name": "has quality: does not say which it lack adequate evidence or it believes to be false."}, "NonQuality": {"name": "no quality: says which it lacks evidence or it believes is false."}},
                "input": "radio",
            },
        },
        "JOB_2": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"HasQuantity": {"name": "has quantity: is as informative as is required"}, "NoQuantity": {"name": "no  quantity: is more or less informative than is required"}},
                "input": "radio",
            },
        },
        "JOB_3": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"HasManner": {"name": "has manner: avoids vagueness, avoids ambiguty, do not play with words, is orderly"}, "NoManner": {"name": "no manner: is vague or ambiguous or plays with words or is not orderly"}},
                "input": "radio",
            },
        },
        "JOB_4": {
            "mlTask": "CLASSIFICATION",
            "required": 1,
            "isChild": False,
            "content": {
                "categories": {"IsPolite": {"name": "is polite"}, "NotPolite": {"name": "is not polite"}},
                "input": "radio",
            },
        }
    }
}

project = kili.create_project(
    title="Testing 2...",
    description="Project Description",
    input_type="TEXT",
    json_interface=interface,
)
project_id = project['id']
print("Project ID: ", project_id)
assets = kili.append_many_to_dataset(
    project_id=project_id,
    content_array = [('comment: ' + d['comment'] + '\n' + 'reply: ' + d['reply']) for d in random_pairs[:3]],
    external_id_array = [d['global_id'] for d in random_pairs[:3]]
)

Project ID:  clp5nrlic1ttw086r5wneguxa


In [ ]:
import random


# Define the sample size (e.g., 3 for a sample of 3 elements)
sample_size = 1

# Use random.sample to get a random sample of the specified size
random_sample = random.sample(conversations_tree, sample_size)
from anytree import PreOrderIter

# Assuming 'root' is the root node of the conversation tree
for node in PreOrderIter(random_sample[0]):
    if node.parent is not None:
        print(f"{node.name}: {node.body}")

random_sample

iqmyeuo: Hey, we launched a new music-scanning feature at Soundslice just a few days ago ([see here](https://www.soundslice.com/blog/226/pdf-and-photo-scanning-beta/)). It uses artificial intelligence to convert PDFs/images into editable sheet music.

I'm happy to run a PDF through it for you if you'd like — just send me a DM!


[Node('/xsyewi', body="Has Anyone Tried Scanscore ?I've been trying to find a pdf scanner app or program that can scan sheet music and turn it into a **musicxml** format. I came across Scanscore, but I don't if it's any good. anyone in this sub try it? would you recommend it?")]

In [ ]:
!pip install  kili

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.9/209.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.0 MB/s eta 0:00:00
  Created wheel for cuid: filename=cuid-0.4-py2.py3-none-any.whl size=4712 sha256=b59c7136e6a0c35c44357a37f14df5f03d3a688b3754ee601d531698bd5923bd
  Stored in directory: /root/.cache/pip/wheels/52/0a/dc/a0e28c435d5a74d9eef3d7c3cd147b96cb21e71e5ec7dcfdbe
Successfully built cuid
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the

In [ ]:
from anytree import Node, RenderTree

# Create a list to store parent-child pairs
parent_child_pairs = []

# Access child nodes from the parent node and add the pairs to the list
for parent in root.children:
    for child in parent.children:
        parent_child_pairs.append((parent.name, child.name))

# Print the parent-child pairs
for pair in parent_child_pairs:
    print(f"Parent: {pair[0]}, Child: {pair[1]}")

# Print the tree structure
for pre, fill, node in RenderTree(root):
    print(f"{pre}{node.name}")


Parent: iqmum2s, Child: iqmunnz
Parent: iqmum2s, Child: iqmuns8
Parent: iqon7tc, Child: iqrmqmy
xsy76f
├── iqmum2s
│   ├── iqmunnz
│   └── iqmuns8
├── iqob6q0
└── iqon7tc
    └── iqrmqmy
        └── iqsvfoq


In [ ]:
import json

for comment in flat_comments_list:


Data has been prepared for Kili labeling.


In [ ]:
formatted_data = []
for comment in flat_comments_list:
    formatted_comment = {
        "row_data": comment['body'],
        "global_key": comment['id'],
        "media_type": "TEXT",
        "metadata_fields": [],
        "attachments": []
    }
    formatted_data.append(formatted_comment)

In [ ]:
with open(base_path +"flat_comments_list.json", "w") as json_file:
    json.dump(flat_comments_list, json_file, indent=2)

print("Data has been written to formatted_comments.json")


Data has been written to formatted_comments.json


In [ ]:
conversations_tree.index(0)

ValueError: ignored

In [ ]:
def flatten_tree(node, flattened_tree):
    flattened_tree.append(node)
    for child in node.children:
        flatten_tree(child, flattened_tree)

In [ ]:
flattened_tree = []
flatten_tree(root, flattened_tree)

In [ ]:
flatten_tree

<function __main__.flatten_tree(node, flattened_tree)>

In [ ]:
# there is node without body!

In [ ]:
from convokit import Corpus, Utterance, Speaker
# Create a Corpus object from the flattened tree.
corpus = Corpus(utterances=[
        Utterance(id=node['id'], text=node['body'], speaker=Speaker(id = node['author']), reply_to=node['reply_to'], conversation_id=node['conversation_id'])
        for node in flat_comments_list
])

In [ ]:
corpus = corpus.fit(corpus)

AttributeError: ignored

In [ ]:
counter = 0
for convo in corpus.iter_conversations():
    counter+=1
print(counter)

2940


In [ ]:
corpus.print_summary_stats()

Number of Speakers: 20381
Number of Utterances: 37647
Number of Conversations: 2940


In [ ]:
from convokit.text_processing import TextParser
parser = TextParser(verbosity=100)
corpus = parser.transform(corpus)

100/37647 utterances processed
200/37647 utterances processed
300/37647 utterances processed
400/37647 utterances processed
500/37647 utterances processed
600/37647 utterances processed
700/37647 utterances processed
800/37647 utterances processed
900/37647 utterances processed
1000/37647 utterances processed
1100/37647 utterances processed
1200/37647 utterances processed
1300/37647 utterances processed
1400/37647 utterances processed
1500/37647 utterances processed
1600/37647 utterances processed
1700/37647 utterances processed
1800/37647 utterances processed
1900/37647 utterances processed
2000/37647 utterances processed
2100/37647 utterances processed
2200/37647 utterances processed
2300/37647 utterances processed
2400/37647 utterances processed
2500/37647 utterances processed
2600/37647 utterances processed
2700/37647 utterances processed
2800/37647 utterances processed
2900/37647 utterances processed
3000/37647 utterances processed
3100/37647 utterances processed
3200/37647 uttera

In [ ]:
from convokit import politenessStrategies
ps = politenessStrategies.PolitenessStrategies(verbose = 1000)
corpus = ps.fit_transform(corpus=corpus)

1000/37647 utterances processed
2000/37647 utterances processed
3000/37647 utterances processed
4000/37647 utterances processed
5000/37647 utterances processed
6000/37647 utterances processed
7000/37647 utterances processed
8000/37647 utterances processed
9000/37647 utterances processed
10000/37647 utterances processed
11000/37647 utterances processed
12000/37647 utterances processed
13000/37647 utterances processed
14000/37647 utterances processed
15000/37647 utterances processed
16000/37647 utterances processed
17000/37647 utterances processed
18000/37647 utterances processed
19000/37647 utterances processed
20000/37647 utterances processed
21000/37647 utterances processed
22000/37647 utterances processed
23000/37647 utterances processed
24000/37647 utterances processed
25000/37647 utterances processed
26000/37647 utterances processed
27000/37647 utterances processed
28000/37647 utterances processed
29000/37647 utterances processed
30000/37647 utterances processed
31000/37647 utteran

In [ ]:
ps.summarize(corpus=corpus)

feature_politeness_==Please==                 0.015220
feature_politeness_==Please_start==           0.028183
feature_politeness_==HASHEDGE==               0.198528
feature_politeness_==Indirect_(btw)==         0.000186
feature_politeness_==Hedges==                 0.084841
feature_politeness_==Factuality==             0.058889
feature_politeness_==Deference==              0.014822
feature_politeness_==Gratitude==              0.041783
feature_politeness_==Apologizing==            0.008739
feature_politeness_==1st_person_pl.==         0.068956
feature_politeness_==1st_person==             0.251999
feature_politeness_==1st_person_start==       0.201291
feature_politeness_==2nd_person==             0.233724
feature_politeness_==2nd_person_start==       0.053789
feature_politeness_==Indirect_(greeting)==    0.009669
feature_politeness_==Direct_question==        0.050416
feature_politeness_==Direct_start==           0.088188
feature_politeness_==HASPOSITIVE==            0.413393
feature_po

In [ ]:
from anytree import Node, find
corpus.get_utterance("iqp0hc8")
target_node = find(conversations_tree, lambda node: node.name == "iqp0hc8")
print(target_node.body)

AttributeError: ignored

In [ ]:
corpus.get_conversation("xsyg0d").meta

ConvoKitMeta({})

In [ ]:
def calculate_conversation_score(dataset_with_metrics_score, metrics_weights):
  i = 2

In [ ]:
file_path = base_path + 'train.tsv.tar.gz'

In [ ]:
import tarfile
import gzip
import pandas as pd

# Specify the path to the .tsv.tar.gz file

# Open the .tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract the .tsv.gz file from the .tar.gz archive
    tsv_file = tar.getmembers()[0]
    file_extension = tsv_file.name.split('.')[-1]

    # Check if the file is compressed
    if file_extension == 'gz':
        # Open the .tsv.gz file
        with gzip.open(tar.extractfile(tsv_file), 'rt') as tsv:
            # Read the lines and store them in a list
            lines = [line.strip() for line in tsv]

    else:
        # Open the uncompressed .tsv file
        with tar.extractfile(tsv_file) as tsv:
            # Read the lines and store them in a list
            lines = [line.decode().strip() for line in tsv]

# Create a DataFrame from the lines
df = pd.DataFrame([line.split('\t') for line in lines])

# Optionally, you can set the column names based on the first row
df.columns = df.iloc[0]
df = df[1:]  # Remove the first row (column names)

# Print the DataFrame
print(df)


KeyboardInterrupt: ignored

In [ ]:
import tarfile
import gzip
import pandas as pd

# Specify the path to the .tsv.tar.gz file
#file_path = 'path/to/your/file.tsv.tar.gz'

# Specify the number of lines to read at a time
batch_size = 1000

# Open the .tar.gz file
with tarfile.open(file_path, 'r:gz') as tar:
    # Extract the .tsv.gz file from the .tar.gz archive
    tsv_file = tar.getmembers()[0]
    file_extension = tsv_file.name.split('.')[-1]

    # Check if the file is compressed
    if file_extension == 'gz':
        # Open the .tsv.gz file
        with gzip.open(tar.extractfile(tsv_file), 'rt') as tsv:
            # Read the lines in batches
            lines = []
            for line in tsv:
                lines.append(line.strip())
                if len(lines) == batch_size:
                    # Process the lines and create a DataFrame
                    df = pd.DataFrame([line.split('\t') for line in lines])

                    # Optionally, set the column names based on the first row
                    df.columns = df.iloc[0]
                    df = df[1:]  # Remove the first row (column names)

                    # Do further processing with the DataFrame
                    # ...

                    # Clear the lines list to free memory
                    lines = []

            # Process the remaining lines
            if lines:
                df = pd.DataFrame([line.split('\t') for line in lines])
                df.columns = df.iloc[0]
                df = df[1:]

    else:
        # Open the uncompressed .tsv file
        with tar.extractfile(tsv_file) as tsv:
            # Read the lines in batches
            lines = []
            for line in tsv:
                lines.append(line.decode().strip())
                if len(lines) == batch_size:
                    # Process the lines and create a DataFrame
                    df = pd.DataFrame([line.split('\t') for line in lines])

                    # Optionally, set the column names based on the first row
                    df.columns = df.iloc[0]
                    df = df[1:]  # Remove the first row (column names)

                    # Do further processing with the DataFrame
                    # ...
                    print(df.columns)
                    # Clear the lines list to free memory
                    lines = []

            # Process the remaining lines
            if lines:
                df = pd.DataFrame([line.split('\t') for line in lines])
                df.columns = df.iloc[0]
                df = df[1:]

# Print the DataFrame
#print(df)
